In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


# Data prepocessing

In [2]:
text_df = pd.read_csv('dataset/multimodal variables - Text-LIWC_Gemini-1.5-pro.csv').sort_values('video')
text_var = text_df[['self-confidence','creativity','enthusiasm','future-orientation','trustworthiness','proactive-personality']]
audio_df = pd.read_csv('dataset/multimodal variables - Audio-Sonix.csv').sort_values('video')
audio_var = audio_df[['assertiveness',	'creativity','enthusiasm',	'future-orientation',	'optimism',	'proactive-personality','self-confidence']]
video_df = pd.read_csv('dataset/multimodal variables - Video-Gemini-1.5-Pro.csv').sort_values('video')
video_var = video_df[['self-confidence',	'creativity','enthusiasm',	'trustworthiness',	'optimism',	'proactive-personality','future-orientation']]
startup_df = pd.read_csv('dataset/Startup_New_backUP.xlsx - Sheet1.csv')
original_var = startup_df[['num_founders','Priorsharedexperience','Industryexperience','educationalqualification']]
original_var = original_var.apply(pd.to_numeric, errors='coerce')
y_var = startup_df[['Fundraised','Backers']]
y_var['Fundraised'] = y_var['Fundraised'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)
y_var['Backers'] = y_var['Backers'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)

/var/folders/t9/x5knz49s7qj_dry4y1r799x00000gn/T/ipykernel_91368/2183331632.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_var['Fundraised'] = y_var['Fundraised'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)
/var/folders/t9/x5knz49s7qj_dry4y1r799x00000gn/T/ipykernel_91368/2183331632.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_var['Backers'] = y_var['Backers'].copy().apply(lambda x: x.replace('$','').replace(',','')).astype(float)


In [3]:
for var_df in [original_var, text_var, audio_var, video_var]:
    #impute missing values with mean
    
    var_df.fillna(var_df.mean(), inplace=True)

y_fundraised = y_var['Fundraised']
y_backers = y_var['Backers']

/var/folders/t9/x5knz49s7qj_dry4y1r799x00000gn/T/ipykernel_91368/505711076.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_df.fillna(var_df.mean(), inplace=True)


In [4]:
def significance_stars(p):
    if p < 0.01:
        return "***"
    elif p < 0.05:
        return "**"
    elif p < 0.1:
        return "*"
    else:
        return ""


def output_result(model, file_name):
    summary_df = pd.DataFrame({
        "Variable": model.params.index,
        "Coefficient": model.params.values,
        "Std. Error": model.bse.values,
        "t-Statistic": model.tvalues.values,
        "p-Value": model.pvalues.values,
    })

    summary_df = summary_df.round(4)


    summary_df["Significance"] = summary_df["p-Value"].apply(significance_stars)

    # Add header
    summary_df.rename(columns={
        "Coefficient": "Coef.",
        "Std. Error": "Std.Err.",
        "t-Statistic": "t-Stat",
        "p-Value": "P>|t|",
    }, inplace=True)

    summary_df.to_csv(f"reg_results/{file_name}.csv", index=False)
    print(summary_df)

# Model 0(Benchmark)

In [5]:
# regression with basic variables
# FUNDRAISED
X = pd.DataFrame(original_var)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
output_result(model, "fundraised_basic")

                   Variable       Coef.   Std.Err.  t-Stat   P>|t|  \
0                     const  13145.9307  4322.9186  3.0410  0.0029   
1              num_founders   1447.3241  2444.7254  0.5920  0.5550   
2     Priorsharedexperience  -1762.6033  8128.0453 -0.2169  0.8287   
3        Industryexperience   5134.2715  5562.6230  0.9230  0.3580   
4  educationalqualification   5291.3242  2207.6905  2.3968  0.0182   

  Significance  
0          ***  
1               
2               
3               
4           **  


In [52]:
model = sm.OLS(y_backers, X).fit()
output_result(model, "backers_basic")

                   Variable     Coef.  Std.Err.  t-Stat   P>|t| Significance
0                     const  193.6718   59.5293  3.2534  0.0015          ***
1              num_founders   34.0676   33.6654  1.0119  0.3137             
2     Priorsharedexperience  -24.9210  111.9283 -0.2227  0.8242             
3        Industryexperience   87.9539   76.6008  1.1482  0.2533             
4  educationalqualification   65.7736   30.4013  2.1635  0.0326           **


# Model 1(Text Only)

In [53]:
# regression with text variables without original variables on fundraised

X = pd.concat([text_var], axis=1)
X = sm.add_constant(X)
y_fundraised = y_var['Fundraised']

model = sm.OLS(y_fundraised, X).fit()
output_result(model, "fundraised_text")

                Variable       Coef.    Std.Err.  t-Stat   P>|t| Significance
0                  const  19244.6991  22911.7719  0.8399  0.4028             
1        self-confidence      6.7892     90.0553  0.0754  0.9400             
2             creativity   3874.0730   1607.9716  2.4093  0.0176           **
3             enthusiasm   -233.1434    469.8096 -0.4963  0.6207             
4     future-orientation  -3622.6905   3048.0067 -1.1885  0.2372             
5        trustworthiness  -1577.5434   1213.2310 -1.3003  0.1962             
6  proactive-personality  -1245.3370   2434.9901 -0.5114  0.6101             


In [54]:
# regression with text variables without original variables on backers
y_backers = y_var['Backers']
model = sm.OLS(y_backers, X).fit()
output_result(model, "backers_text")

                Variable     Coef.  Std.Err.  t-Stat   P>|t| Significance
0                  const  186.4009  319.5085  0.5834  0.5608             
1        self-confidence    0.3355    1.2558  0.2671  0.7899             
2             creativity   32.1740   22.4234  1.4348  0.1542             
3             enthusiasm   -4.9199    6.5516 -0.7510  0.4543             
4     future-orientation  -72.8670   42.5050 -1.7143  0.0893            *
5        trustworthiness  -10.7113   16.9187 -0.6331  0.5280             
6  proactive-personality    5.2913   33.9563  0.1558  0.8765             


In [55]:
# regression with text variables and original variables on fundraised

X = pd.concat([text_var,original_var], axis=1)
X = sm.add_constant(X)
y_fundraised = y_var['Fundraised']

model = sm.OLS(y_fundraised, X).fit()
output_result(model, "fundraised_text_original")

                    Variable       Coef.    Std.Err.  t-Stat   P>|t|  \
0                      const  17167.3725  23094.6711  0.7433  0.4589   
1            self-confidence     -8.8379     92.0938 -0.0960  0.9237   
2                 creativity   3709.0593   1610.8853  2.3025  0.0233   
3                 enthusiasm   -459.2056    472.9522 -0.9709  0.3338   
4         future-orientation  -3489.9095   3032.7574 -1.1507  0.2524   
5            trustworthiness  -1757.5261   1206.3163 -1.4569  0.1481   
6      proactive-personality  -1023.3961   2458.7685 -0.4162  0.6781   
7               num_founders   1141.7928   2498.5701  0.4570  0.6486   
8      Priorsharedexperience  -4410.4175   8144.5592 -0.5415  0.5893   
9         Industryexperience   5538.6892   5584.1410  0.9919  0.3235   
10  educationalqualification   5445.0595   2268.7908  2.4000  0.0181   

   Significance  
0                
1                
2            **  
3                
4                
5                
6        

In [56]:
# regression with text variables with original variables on backers
y_backers = y_var['Backers']
model = sm.OLS(y_backers, X).fit()
output_result(model, "backers_text_original")

                    Variable     Coef.  Std.Err.  t-Stat   P>|t| Significance
0                      const  118.8443  322.0382  0.3690  0.7128             
1            self-confidence    0.0073    1.2842  0.0057  0.9955             
2                 creativity   29.4547   22.4626  1.3113  0.1926             
3                 enthusiasm   -7.7442    6.5950 -1.1743  0.2429             
4         future-orientation  -70.4651   42.2896 -1.6663  0.0986            *
5            trustworthiness  -12.6040   16.8212 -0.7493  0.4553             
6      proactive-personality   10.3382   34.2857  0.3015  0.7636             
7               num_founders   33.1775   34.8407  0.9523  0.3431             
8      Priorsharedexperience  -68.4842  113.5699 -0.6030  0.5478             
9         Industryexperience   89.1329   77.8667  1.1447  0.2549             
10  educationalqualification   68.9404   31.6366  2.1791  0.0315           **


# Model2 (Text+Audio)

In [57]:
# regression with text + audio variables without original variables on backers

X = pd.concat([text_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['trustworthiness','proactive-personality']]], axis=1)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
output_result(model, "fundraised_text_audio")

                Variable       Coef.    Std.Err.  t-Stat   P>|t| Significance
0                  const  16506.9046  28335.8123  0.5825  0.5614             
1             creativity   2394.5678   1771.2579  1.3519  0.1792             
2     future-orientation  -3585.8829   3005.6347 -1.1931  0.2354             
3        self-confidence   5640.8536   2813.6711  2.0048  0.0474           **
4             enthusiasm  -4032.2809   2929.1062 -1.3766  0.1714             
5        trustworthiness  -1526.3868   1186.4220 -1.2865  0.2010             
6  proactive-personality  -1222.2715   2389.6869 -0.5115  0.6100             


In [58]:
# regression with text + audio variables without original variables on backers
model = sm.OLS(y_backers, X).fit()
output_result(model, "backers_text_audio")

                Variable     Coef.  Std.Err.  t-Stat   P>|t| Significance
0                  const  239.8386  402.3781  0.5961  0.5524             
1             creativity   25.7891   25.1525  1.0253  0.3075             
2     future-orientation  -68.9854   42.6810 -1.6163  0.1089             
3        self-confidence   21.7230   39.9551  0.5437  0.5878             
4             enthusiasm  -23.8297   41.5943 -0.5729  0.5679             
5        trustworthiness  -11.7504   16.8476 -0.6975  0.4870             
6  proactive-personality    5.5590   33.9344  0.1638  0.8702             


In [59]:
# regression with text + audio variables and original variables on fundraised

X = pd.concat([text_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['trustworthiness','proactive-personality']], original_var], axis=1)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
output_result(model, "fundraised_text_audio_original")

                    Variable       Coef.    Std.Err.  t-Stat   P>|t|  \
0                      const  15452.8952  28491.6151  0.5424  0.5887   
1                 creativity   2452.5519   1787.1247  1.3723  0.1729   
2         future-orientation  -3281.3804   3015.2594 -1.0883  0.2789   
3            self-confidence   4701.2842   2854.8324  1.6468  0.1026   
4                 enthusiasm  -3566.4442   2965.3014 -1.2027  0.2318   
5            trustworthiness  -1781.4609   1192.4915 -1.4939  0.1382   
6      proactive-personality  -1074.0696   2417.9384 -0.4442  0.6578   
7               num_founders    842.4417   2451.6531  0.3436  0.7318   
8      Priorsharedexperience  -4335.7482   8114.6852 -0.5343  0.5942   
9         Industryexperience   4115.8319   5603.4708  0.7345  0.4643   
10  educationalqualification   4732.4773   2203.6022  2.1476  0.0340   

   Significance  
0                
1                
2                
3                
4                
5                
6        

In [60]:
# regression with text + audio variables and original variables on backers
model = sm.OLS(y_backers, X).fit()
output_result(model, "backers_text_audio_original")

                    Variable     Coef.  Std.Err.  t-Stat   P>|t| Significance
0                      const  188.2093  402.9897  0.4670  0.6414             
1                 creativity   28.0453   25.2774  1.1095  0.2697             
2         future-orientation  -63.1121   42.6483 -1.4798  0.1419             
3            self-confidence    3.5676   40.3792  0.0884  0.9298             
4                 enthusiasm  -16.4117   41.9417 -0.3913  0.6964             
5            trustworthiness  -14.5571   16.8668 -0.8631  0.3901             
6      proactive-personality    9.0926   34.1997  0.2659  0.7909             
7               num_founders   33.3443   34.6765  0.9616  0.3384             
8      Priorsharedexperience  -60.7340  114.7753 -0.5292  0.5978             
9         Industryexperience   89.4487   79.2563  1.1286  0.2616             
10  educationalqualification   61.4911   31.1681  1.9729  0.0511            *


# Model3 (Text+Audio+Video)

In [61]:
# regression with text + audio+video variables without original variables on fundraised

X = pd.concat([video_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['proactive-personality','trustworthiness']]], axis=1)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
output_result(model, "fundraised_text_audio_video")

                Variable      Coef.    Std.Err.  t-Stat   P>|t| Significance
0                  const -6042.2403  33718.9072 -0.1792  0.8581             
1             creativity  1734.4685   1835.2043  0.9451  0.3467             
2     future-orientation   941.3499   1784.6187  0.5275  0.5989             
3        self-confidence  7241.8159   2603.7535  2.7813  0.0064          ***
4             enthusiasm -4896.7297   3008.1780 -1.6278  0.1064             
5  proactive-personality  -525.5407   2307.0424 -0.2278  0.8202             
6        trustworthiness -1556.9181   1189.8571 -1.3085  0.1934             


In [62]:
# regression with text + audio+video variables without original variables on backers
model = sm.OLS(y_backers, X).fit()
output_result(model, "backers_text_audio_video")

                Variable    Coef.  Std.Err.  t-Stat   P>|t| Significance
0                  const  33.5450  479.4961  0.0700  0.9444             
1             creativity  28.4346   26.0973  1.0896  0.2783             
2     future-orientation  -2.6930   25.3780 -0.1061  0.9157             
3        self-confidence  35.9018   37.0264  0.9696  0.3344             
4             enthusiasm -30.9510   42.7775 -0.7235  0.4709             
5  proactive-personality   9.9954   32.8071  0.3047  0.7612             
6        trustworthiness -13.2842   16.9202 -0.7851  0.4341             


In [63]:
# regression with text + audio+video variables and original variables on fundraised

X = pd.concat([video_var[['creativity','future-orientation']], audio_var[['self-confidence','enthusiasm']],text_var[['proactive-personality','trustworthiness']], original_var], axis=1)
X = sm.add_constant(X)

model = sm.OLS(y_fundraised, X).fit()
output_result(model, "fundraised_text_audio_video_original")

                    Variable      Coef.    Std.Err.  t-Stat   P>|t|  \
0                      const -6307.3485  33937.0775 -0.1859  0.8529   
1                 creativity  1631.9541   1861.2232  0.8768  0.3826   
2         future-orientation   736.7201   1797.9578  0.4098  0.6828   
3            self-confidence  6298.0149   2649.3649  2.3772  0.0192   
4                 enthusiasm -4261.3773   3067.8704 -1.3890  0.1677   
5      proactive-personality  -217.0023   2321.2639 -0.0935  0.9257   
6            trustworthiness -1874.5006   1200.0292 -1.5620  0.1213   
7               num_founders  1075.0595   2473.4283  0.4346  0.6647   
8      Priorsharedexperience -1964.4752   8313.5201 -0.2363  0.8137   
9         Industryexperience  1815.0855   5678.4231  0.3196  0.7499   
10  educationalqualification  5171.6613   2215.0427  2.3348  0.0214   

   Significance  
0                
1                
2                
3            **  
4                
5                
6                
7  

In [64]:
# regression with text + audio+video variables and original variables on backers

model = sm.OLS(y_backers, X).fit()
output_result(model, "backers_text_audio_video_original")

                    Variable    Coef.  Std.Err.  t-Stat   P>|t| Significance
0                      const   9.2633  480.1399  0.0193  0.9846             
1                 creativity  26.8410   26.3325  1.0193  0.3104             
2         future-orientation  -8.8412   25.4374 -0.3476  0.7289             
3            self-confidence  16.9326   37.4831  0.4517  0.6524             
4                 enthusiasm -18.8316   43.4041 -0.4339  0.6653             
5      proactive-personality  16.3959   32.8411  0.4992  0.6186             
6            trustworthiness -16.5768   16.9779 -0.9764  0.3311             
7               num_founders  37.9097   34.9939  1.0833  0.2811             
8      Priorsharedexperience -16.0254  117.6192 -0.1362  0.8919             
9         Industryexperience  67.0843   80.3380  0.8350  0.4056             
10  educationalqualification  68.2937   31.3383  2.1792  0.0315           **
